In [209]:
import pandas
import urllib
from selenium import webdriver
from time import sleep
import re
import os.path
import io
from bs4 import BeautifulSoup

gpo=webdriver.Chrome()
gpo.get("https://www.gpo.gov/fdsys/browse/collection.action?collectionCode=CPD")

In [49]:
years=gpo.find_elements_by_xpath(".//div[@class='level1 browse-level']/a")

years

[<selenium.webdriver.remote.webelement.WebElement (session="a9fdfa3dfa9cd2b4d5feba6f7605e094", element="0.8569822999146735-1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a9fdfa3dfa9cd2b4d5feba6f7605e094", element="0.8569822999146735-2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a9fdfa3dfa9cd2b4d5feba6f7605e094", element="0.8569822999146735-3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a9fdfa3dfa9cd2b4d5feba6f7605e094", element="0.8569822999146735-4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a9fdfa3dfa9cd2b4d5feba6f7605e094", element="0.8569822999146735-5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a9fdfa3dfa9cd2b4d5feba6f7605e094", element="0.8569822999146735-6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a9fdfa3dfa9cd2b4d5feba6f7605e094", element="0.8569822999146735-7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a9fdfa3dfa9cd2b4d5feba6f7605e094", el

In [201]:
def find_all_more(url_var):
    """Finds all text links on a page"""
    return url_var.find_elements_by_partial_link_text('More')

def get_doc_link(links):
    """Takes a list of text links and returns the url value of the actual document"""
    href = []
    for i in range(len(links)):
        href.append(links[i].get_attribute('href'))
    return href

In [202]:
text_file = '/Users/machine72/Desktop/Executive Order Clinton.xml'
with open(text_file, 'r') as text_file:
    text = text_file.read()
print(text)

<?xml version="1.0" encoding="UTF-8"?><mods xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:exslt="http://exslt.org/common" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.loc.gov/mods/v3" version="3.3" xsi:schemaLocation="http://www.loc.gov/mods/v3 http://www.loc.gov/standards/mods/v3/mods-3-3.xsd" ID="id-WCPD-1997-10-20-Pg1567">
<name type="corporate">
<namePart>United States Government Publishing Office</namePart>
<role>
<roleTerm authority="marcrelator" type="text">publisher</roleTerm>
<roleTerm authority="marcrelator" type="code">pbl</roleTerm>
</role>
<role>
<roleTerm authority="marcrelator" type="text">distributor</roleTerm>
<roleTerm authority="marcrelator" type="code">dst</roleTerm>
</role>
</name>
<name type="corporate">
<namePart>United States</namePart>
<namePart>National Archives and Records Administration</namePart>
<namePart>Office of the Federal Register</namePart>
<role>
<roleTerm authority="marcrelator" type="text">author</roleTerm>
<roleTerm 

In [203]:
presidents = ['"GWB"', '"BHO"', '"DJT"', '"WJC"']
names = ['George W. Bush', 'Barack H. Obama', 'Donald J. Trump', 'William J. Clinton']
parties = ['"R"', '"D"' ]
party_name = ['Republican', 'Democrat']

President

In [204]:
def president(text):
    pres = re.findall('(?<=president id=).*\w\S', text)
    x = pres[0].split()
    for president in presidents: 
        if president in x: 
            return names[presidents.index(president)]
president(text)

'William J. Clinton'

Party

In [205]:
def party(text):
    par = re.findall('(?<=party=).*\w\S', text)
    for party in parties: 
        if party in par:
            return party_name[parties.index(party)]
party(text)

'Democrat'

Statement

In [206]:
def statement(text):
    soup = BeautifulSoup(text, 'xml')
    tags = soup.find('title')
    return tags.text

statement(text)

'Executive Order 13064--Further Amendment to Executive Order 13010, as Amended, Critical Infrastructure Protection'

Date

In [207]:
def date(txt):
    date_issued = re.findall(r'<dateIssued .*>(.+?)<\/dateIssued>',txt)
    return date_issued[0]
date(text)

'1997-10-20'

Notes

In [208]:
def note(txt):
    soup = BeautifulSoup(txt, 'xml')
    tags = [a for a in (td.findAll('title') for td in soup.findAll('titleInfo')) if a]
    if not tags:
        return "None"
    flatten_list = [x for y in tags for x in y]
    tags2 = [BeautifulSoup(str(i), "xml") for i in flatten_list]
    no_tags = [i.text for i in tags2]
    return no_tags[0]

note(text)


'Executive Order 13064--Further Amendment to Executive Order 13010, as Amended, Critical Infrastructure Protection'

Categories

In [196]:
def categories(txt): 
    soup = BeautifulSoup(text, 'xml')
    tags = [a for a in (td.find('dcpdCategory') for td in soup.findAll('extension')) if a]
    if not tags:
        return 'None'
    tags = BeautifulSoup(str(tags[0]),'xml')
    return print(tags.text)

categories(text)

'None'

Subjects

In [200]:
def subjects(txt):
    soup = BeautifulSoup(text, 'xml')
    tags = [a for a in (td.findAll('topic') for td in soup.findAll('subject')) if a] 
    if not tags:
        return "None"
    flatten_list = [x for y in tags for x in y]
    tags2 = [BeautifulSoup(str(i), "xml") for i in flatten_list]
    no_tags = [i.text for i in tags2]
    return no_tags[0]

subjects(text)

'Executive Orders'

DCPD

In [198]:
def dcpd(txt) :
    dcpd = re.findall(r'<accessId>(.+?)</accessId>',txt)
    dcpd2 =  re.sub("\D", "", dcpd[0])
    return dcpd2
dcpd(text)

'199710201567'

In [199]:
#Works for Trump
#Works for Obama
#Works for Bush
#Works for Clinton